In [1]:
import csv
import random


In [2]:
class User:
    def __init__(self, prolific_id, age, gender, height, hair_type, hair_color, has_tattoos,
                 education, is_student, is_in_workforce, industry,
                 hobbies, favorite_hobby, interests, favorite_interest, music_genre, frequent_locations,
                 bar_day_of_week, bar_time_of_day, bar_group_size, bar_clothing,
                 restaurant_day_of_week, restaurant_time_of_day, restaurant_group_size, restaurant_clothing,
                 university_day_of_week, university_time_of_day, university_group_size, university_clothing,
                 workplace_day_of_week, workplace_time_of_day, workplace_group_size, workplace_clothing,
                 community_day_of_week, community_time_of_day, community_group_size, community_clothing,
                 cafe_day_of_week, cafe_time_of_day, cafe_group_size, cafe_clothing,
                 attraction_day_of_week, attraction_time_of_day, attraction_group_size, attraction_clothing,
                 outdoor_day_of_week, outdoor_time_of_day, outdoor_group_size, outdoor_clothing,
                 gym_day_of_week, gym_time_of_day, gym_group_size, gym_clothing,
                 personality, listen_or_speak, social_media, favorite_social_media, music_listen_time):

        self.prolific_id = prolific_id
        self.age = age
        self.gender = gender
        self.height = height
        self.hair_type = hair_type
        self.hair_color = hair_color
        self.has_tattoos = has_tattoos
        self.education = education
        self.is_student = is_student
        self.is_in_workforce = is_in_workforce
        self.industry = industry
        self.hobbies = hobbies.split(';')
        self.favorite_hobby = favorite_hobby
        self.interests = interests.split(';')
        self.favorite_interest = favorite_interest
        self.music_genre = music_genre
        self.frequent_locations = frequent_locations.split(';')

        # Location #1: Bar.
        self.bar_day_of_week = bar_day_of_week
        self.bar_time_of_day = bar_time_of_day
        self.bar_group_size = bar_group_size
        self.bar_clothing = bar_clothing.split(';')

        # Location #2: Restaurant.
        self.restaurant_day_of_week = restaurant_day_of_week
        self.restaurant_time_of_day = restaurant_time_of_day
        self.restaurant_group_size = restaurant_group_size
        self.restaurant_clothing = restaurant_clothing.split(';')

        # Location #3: University.
        self.university_day_of_week = university_day_of_week
        self.university_time_of_day = university_time_of_day
        self.university_group_size = university_group_size
        self.university_clothing = university_clothing.split(';')

        # Location #4: Workplace.
        self.workplace_day_of_week = workplace_day_of_week
        self.workplace_time_of_day = workplace_time_of_day
        self.workplace_group_size = workplace_group_size
        self.workplace_clothing = workplace_clothing.split(';')

        # Location #5: Community.
        self.community_day_of_week = community_day_of_week
        self.community_time_of_day = community_time_of_day
        self.community_group_size = community_group_size
        self.community_clothing = community_clothing.split(';')

        # Location #6: Cafe.
        self.cafe_day_of_week = cafe_day_of_week
        self.cafe_time_of_day = cafe_time_of_day
        self.cafe_group_size = cafe_group_size
        self.cafe_clothing = cafe_clothing.split(';')

        # Location #7: Attraction.
        self.attraction_day_of_week = attraction_day_of_week
        self.attraction_time_of_day = attraction_time_of_day
        self.attraction_group_size = attraction_group_size
        self.attraction_clothing = attraction_clothing.split(';')

        # Location #8: Outdoor.
        self.outdoor_day_of_week = outdoor_day_of_week
        self.outdoor_time_of_day = outdoor_time_of_day
        self.outdoor_group_size = outdoor_group_size
        self.outdoor_clothing = outdoor_clothing.split(';')

        # Location #9: Gym.
        self.gym_day_of_week = gym_day_of_week
        self.gym_time_of_day = gym_time_of_day
        self.gym_group_size = gym_group_size
        self.gym_clothing = gym_clothing.split(';')

        self.personality = personality
        self.listen_or_speak = listen_or_speak
        self.social_media = social_media.split(';')
        self.favorite_social_media = favorite_social_media
        self.music_listen_time = music_listen_time


In [3]:
users = []

with open('misc/cohort2_survey1_20_people.csv', 'r') as file:
    reader = csv.reader(file)
    next(reader)
    for row in reader:
        user = User(*row)
        users.append(user)


In [4]:
user = users[1]
print(user.is_student)
print(user.university_day_of_week)
print(("I don't go to university." in user.university_clothing) == ("I don't go to university." in user.university_day_of_week))

Yes.
Weekday (Monday - Friday).
True


In [5]:
import json

"""
append error message to map of rejected or flagged participants
"""
def new_error(rejected_or_flagged, id, err_msg):
  if id in rejected_or_flagged:
    rejected_or_flagged[id].append(err_msg)
  else:
    rejected_or_flagged[id] = [err_msg]

"""
reject participants based on various criteria
"""
def reject_participants(users):
  flagged = {}
  rejected = {}
  not_student_text = "I don't go to university."
  not_workplace_text = "I don't have a workplace."

  for user in users:
    err_msg = "[REJECTED] they said they go to university frequently, but select later that they don't."
    if 'School / University' in user.frequent_locations or \
        'University' in user.frequent_locations: # TODO: added for backward compatibility support, remove later.
      if not_student_text in user.university_clothing or \
          not_student_text in user.university_day_of_week or \
          not_student_text in user.university_group_size or \
          not_student_text in user.university_time_of_day:
        new_error(rejected, user.prolific_id, err_msg)

    err_msg = "[FLAGGED] they said they are a student, but select later that they don't go to university."
    if 'Yes.' in user.is_student:
      if not_student_text in user.university_clothing or \
          not_student_text in user.university_day_of_week or \
          not_student_text in user.university_group_size or \
          not_student_text in user.university_time_of_day:
        new_error(flagged, user.prolific_id, err_msg)

    err_msg = "[REJECTED] they select they don't have a university but they choose clothing for university."
    if not_student_text in user.university_clothing and len(user.university_clothing) > 1:
      new_error(rejected, user.prolific_id, err_msg)

    err_msg = '''[REJECTED] their answers in the location-specific questions should be consistent. \
they should answer "not in university" for all questions or none.'''
    if (not_student_text in user.university_clothing) != (not_student_text in user.university_day_of_week) or \
          (not_student_text in user.university_day_of_week) != (not_student_text in user.university_group_size) or \
          (not_student_text in user.university_group_size) != (not_student_text in user.university_time_of_day):
      new_error(rejected, user.prolific_id, err_msg)

    err_msg = "[REJECTED] they said they go to workplace frequently, but select later that they don't."
    if 'Workplace' in user.frequent_locations:
      if not_workplace_text in user.workplace_clothing or \
          not_workplace_text in user.workplace_day_of_week or \
          not_workplace_text in user.workplace_group_size or \
          not_workplace_text in user.workplace_time_of_day:
        new_error(rejected, user.prolific_id, err_msg)

    err_msg = '''[REJECTED] they said they are not in the workforce, but \
select workplace as a frequently visited location.'''
    if 'No.' in user.is_in_workforce and 'Workplace' in user.frequent_locations:
      new_error(rejected, user.prolific_id, err_msg)

    err_msg = "[FLAGGED] they said they are in the workforce, but select later that they don't have a workplace."
    if 'Yes.' in user.is_in_workforce:
      if not_workplace_text in user.workplace_clothing or \
          not_workplace_text in user.workplace_day_of_week or \
          not_workplace_text in user.workplace_group_size or \
          not_workplace_text in user.workplace_time_of_day:
        new_error(flagged, user.prolific_id, err_msg)

    err_msg = "[REJECTED] they select they don't have a workplace but they choose clothing for workplace."
    if not_workplace_text in user.workplace_clothing and len(user.workplace_clothing) > 1:
      new_error(rejected, user.prolific_id, err_msg)

    err_msg = '''[REJECTED] their answers in the location-specific questions should be consistent. \
they should answer "not in workplace" for all questions or none.'''
    if (not_workplace_text in user.workplace_clothing) != (not_workplace_text in user.workplace_day_of_week) or \
          (not_workplace_text in user.workplace_day_of_week) != (not_workplace_text in user.workplace_group_size) or \
          (not_workplace_text in user.workplace_group_size) != (not_workplace_text in user.workplace_time_of_day):
      new_error(rejected, user.prolific_id, err_msg)

    err_msg = '''[REJECTED] they stated that their furthest in-progess or \
completed degree was high school or GED, even though we filtered users out by \
education level.'''
    if 'High school diploma or GED' in user.education or \
          'Some high school or less' in user.education:
      new_error(rejected, user.prolific_id, err_msg)

    err_msg = '''[FLAGGED] did not list university as frequent location, but \
answered the location-specific questions for university.'''
    if 'University' in user.frequent_locations and (
        not_student_text not in user.university_clothing or \
        not_student_text not in user.university_day_of_week or \
        not_student_text not in user.university_group_size or \
        not_student_text not in user.university_time_of_day
    ):
      new_error(flagged, user.prolific_id, err_msg)

    err_msg = '''[FLAGGED] did not list workplace as frequent location, but \
answered the location-specific questions for workplace.'''
    if 'Workplace' in user.frequent_locations and (
        not_workplace_text not in user.workplace_clothing or \
        not_workplace_text not in user.workplace_day_of_week or \
        not_workplace_text not in user.workplace_group_size or \
        not_workplace_text not in user.workplace_time_of_day
    ):
      new_error(flagged, user.prolific_id, err_msg)

    err_msg = '[FLAGGED] they said that they were working on the weekends, evenings, or nights.'
    if 'Evening' in user.workplace_time_of_day or \
          'Weekend' in user.workplace_time_of_day or \
          'Nights' in user.workplace_time_of_day:
      new_error(flagged, user.prolific_id, err_msg)

    err_msg = '[FLAGGED] they said that their favorite hobby is not one of the hobbies that they currently do.'
    if user.favorite_hobby not in user.hobbies:
      new_error(flagged, user.prolific_id, err_msg)

    err_msg = '[REJECT] they said that their favorite interest is not one of the topics that interest them.'
    if user.favorite_interest not in user.interests:
      new_error(rejected, user.prolific_id, err_msg)

    err_msg = '[FLAGGED] introvert tends to speak more or extrovert tends to listen more.'
    if ('Extrovert' in user.personality and 'listens' in user.listen_or_speak) or \
          ('Introvert' in user.personality and 'speaks' in user.listen_or_speak):
      new_error(flagged, user.prolific_id, err_msg)

    err_msg = '[FLAGGED] their favorite social media is not one of the social medias that they used more often.'
    if user.favorite_social_media not in user.social_media:
      new_error(flagged, user.prolific_id, err_msg)

  print("Rejected:", json.dumps(rejected, indent=4))
  print("Flagged:", json.dumps(flagged, indent=4))
  return rejected

In [6]:
print(len(users))
rejected = reject_participants(users)
for reject in rejected:
  for user in users:
    if user.prolific_id == reject:
      print("removed:", user.prolific_id)
      users.remove(user)
      break
print(len(users))

19
Rejected: {
    "614eacdc9be2d87db76205d3": [
        "[REJECT] they said that their favorite interest is not one of the topics that interest them."
    ],
    "5d8c0aed48b2d10015bfcdb7": [
        "[REJECTED] they stated that their furthest in-progess or completed degree was high school or GED, even though we filtered users out by education level."
    ],
    "66241ad491d663005540ffc5": [
        "[REJECTED] they stated that their furthest in-progess or completed degree was high school or GED, even though we filtered users out by education level."
    ]
}
Flagged: {
    "5e544727059e3744ccdd4c3b": [
        "[FLAGGED] did not list workplace as frequent location, but answered the location-specific questions for workplace."
    ],
    "65d8bf71504a6a75a4d66bad": [
        "[FLAGGED] did not list workplace as frequent location, but answered the location-specific questions for workplace."
    ],
    "660b39ffd8e9b35fd0f563b9": [
        "[FLAGGED] did not list workplace as frequent loc

In [7]:
import statistics
males = []
females = []

for user in users:
  if user.gender == 'Male.':
    males.append(user)
  else:
    females.append(user)

variances = []
variance_dict = {}
for i in range(1000):
  ages = []
  selected_male_users = random.sample(males, 5)
  for user in selected_male_users:
    ages.append(int(user.age))
  variance = statistics.variance(ages)
  variances.append(variance)
  variance_dict[variance] = selected_male_users
max_variance = max(variances)
print(max_variance)
selected_male_users = variance_dict[max_variance]

variances = []
variance_dict = {}
for i in range(1000):
  ages = []
  selected_female_users = random.sample(females, 5)
  for user in selected_female_users:
    ages.append(int(user.age))
  variance = statistics.variance(ages)
  variances.append(variance)
  variance_dict[variance] = selected_female_users
max_variance = max(variances)
print(max_variance)
selected_female_users = variance_dict[max_variance]

selected_users = selected_female_users + selected_male_users
random.shuffle(selected_users)

ages = []
for user in selected_users:
  ages.append(int(user.age))
variance = statistics.variance(ages)
print(variance)

57.2
650.7
354.6222222222222


In [8]:
# previously_selected_users = [
#   # add previously selected users here
#   # to remove them from generated scenarios.
# ]

# len(users)
# selected_users = []
# for user in users:
#   if user.prolific_id not in previously_selected_users:
#     selected_users.append(user)
#     print(user.prolific_id, user.age, user.gender)

# len(selected_users)

In [9]:
def get_random_frequent_location(frequent_locations):
    return random.choice(frequent_locations)


In [10]:
GARMENT_OPTIONS = ["Athletic clothes.", "Casual clothes.", "Trendy clothes.", "Formal clothes.", "Designer clothes."]
EYEWEAR_OPTIONS = ["Eyeglasses.", "Sunglasses."]
WATCH_OPTIONS = ["Luxury watch.", "Smart watch."]
OTHER_OPTIONS = ["Hat.", "Necklace.", "Rings.", "Earrings."]

def get_random_clothing(clothing):
    garment_options = []
    eyewear_options = []
    watch_options = []
    other_options = []

    for garment in GARMENT_OPTIONS:
        if garment in clothing or garment[:-1] in clothing:  # `garment[:-1]` removes the period.
            garment_options.append(garment)
    for eyewear in EYEWEAR_OPTIONS:
        if eyewear in clothing or eyewear[:-1] in clothing:  # `eyewear[:-1]` removes the period.
            eyewear_options.append(eyewear)
    for watch in WATCH_OPTIONS:
        if watch in clothing or watch[:-1] in clothing:  # `watch[:-1]` removes the period.
            watch_options.append(watch)
    for other in OTHER_OPTIONS:
        if other in clothing or other[:-1] in clothing:  # `other[:-1]` removes the period.
            other_options.append(other)

    clothing = []
    if len(garment_options) > 0:
        clothing.append(random.choice(garment_options))
    if len(eyewear_options) > 0:
        clothing.append(random.choice(eyewear_options))
    if len(watch_options) > 0:
        clothing.append(random.choice(watch_options))
    clothing.extend(other_options)

    return clothing


In [11]:
def get_day_time_group_size_and_clothing(user, location):
    if location.endswith('.'):
        location = location[:-1]  # Remove the period.
    match location:
        case "Bar / Nightclub":
            return user.bar_day_of_week, user.bar_time_of_day, user.bar_group_size, get_random_clothing(user.bar_clothing)
        case "Sit-down restaurant":
            return user.restaurant_day_of_week, user.restaurant_time_of_day, user.restaurant_group_size, get_random_clothing(user.restaurant_clothing)
        case "School / University":
            return user.university_day_of_week, user.university_time_of_day, user.university_group_size, get_random_clothing(user.university_clothing)
        case "University":  # TODO: added for backward compatibility support, remove later.
            return user.university_day_of_week, user.university_time_of_day, user.university_group_size, get_random_clothing(user.university_clothing)
        case "Workplace":
            return user.workplace_day_of_week, user.workplace_time_of_day, user.workplace_group_size, get_random_clothing(user.workplace_clothing)
        case "Community event (block-party, social club, hangout, potluck, etc.)":
            return user.community_day_of_week, user.community_time_of_day, user.community_group_size, get_random_clothing(user.community_clothing)
        case "Cafe / Coffee shop":
            return user.cafe_day_of_week, user.cafe_time_of_day, user.cafe_group_size, get_random_clothing(user.cafe_clothing)
        case "Attraction (museum, concert, movie theater, shopping mall, amusement park, etc.)":
            return user.attraction_day_of_week, user.attraction_time_of_day, user.attraction_group_size, get_random_clothing(user.attraction_clothing)
        case "Outdoor activity (walking/hiking trail, biking, park, neighborhood, dog park, etc.)":
            return user.outdoor_day_of_week, user.outdoor_time_of_day, user.outdoor_group_size, get_random_clothing(user.outdoor_clothing)
        case "Gym":
            return user.gym_day_of_week, user.gym_time_of_day, user.gym_group_size, get_random_clothing(user.gym_clothing)
        case _:
            raise Exception("Invalid location.")


In [12]:
def translate_location(location):
    if location.endswith('.'):
        location = location[:-1]  # Remove the period.
    match location:
        case "Bar / Nightclub":
            return 'a bar'
        case "Sit-down restaurant":
            return 'a sit-down restaurant'
        case "School / University":
            return 'your school or university'
        case "University":  # TODO: added for backward compatibility support, remove later.
            return 'your school or university'
        case "Workplace":
            return 'your workplace'
        case "Community event (block-party, social club, hangout, potluck, etc.)":
            return 'a community event'
        case "Cafe / Coffee shop":
            return 'a cafe'
        case "Attraction (museum, concert, movie theater, shopping mall, amusement park, etc.)":
            return 'an attraction'
        case "Outdoor activity (walking/hiking trail, biking, park, neighborhood, dog park, etc.)":
            return 'an outdoor activity'
        case "Gym":
            return 'the gym'
        case _:
            raise Exception("Invalid location.")


In [13]:
def translate_day_of_week(day_of_week):
    match day_of_week:
        case "Weekday (Monday - Friday).":
            return 'on a weekday'
        case "Weekend (Saturday, Sunday).":
            return 'on the weekend'
        case _:
            raise Exception("Invalid day of week.")


In [14]:
def translate_time_of_day(time_of_day):
    match time_of_day:
        case "Morning (before 12 PM).":
            return 'in the morning'
        case "Afternoon (after 12 PM and before 6 PM).":
            return 'in the afternoon'
        case "Evening (after 6 PM and before sundown).":
            return 'in the evening'
        case "Nighttime (after sundown).":
            return 'at night'
        case _:
            raise Exception("Invalid time of day.")


In [15]:
def translate_conversational_intensity(conversational_intensity):
    match conversational_intensity:
        case "alone":
            return conversational_intensity  # Do not alter the string.
        case "I tend to listen more (to what others are saying).":
            return 'they are listening to others in their group'
        case "I tend to speak more.":
            return 'they are speaking to others in their group'
        case _:
            raise Exception("Invalid conversational intensity.")


In [16]:
def translate_human_congestion_level(human_congestion_level):
    match human_congestion_level:
        case "crowded":
            return True
        case "not crowded":
            return False

In [17]:
def translate_human_noise_level(human_noise_level):
  match human_noise_level:
    case "There are many people talking":
      return "loud"
    case "There are not many people talking":
      return "quiet"

In [18]:
def translate_non_human_noise_level(non_human_noise_level):
  match non_human_noise_level:
    case "music is playing":
      return "loud"
    case "no music is playing":
      return "quiet"

In [19]:
def translate_gaze_self_to_candidate(gaze_self_to_candidate):
  match gaze_self_to_candidate:
    case "looking":
      return True
    case "not looking":
      return False


In [20]:
def translate_gaze_candidate_to_self(gaze_candidate_to_self):
  match gaze_candidate_to_self:
    case "looking":
      return True
    case "not looking":
      return False

In [21]:
def translate_proximity(proximity):
  match proximity:
    case "far away":
      return "far"
    case "nearby":
      return "near"

In [22]:
class ScenarioInfo:
    def __init__(self,
                 scenario_id,
                 candidate_prolific_id,
                 location,
                 weather,
                 human_congestion_level,
                 human_noise_level,
                 non_human_noise_level,
                 candidate_occluded,
                 gaze_self_to_candidate,
                 gaze_candidate_to_self,
                 proximity,
                 candidate_clothing):
        self.scenario_id = scenario_id
        self.candidate_prolific_id = candidate_prolific_id
        self.location = location
        self.weather = weather
        self.human_congestion_level = human_congestion_level
        self.human_noise_level = human_noise_level
        self.non_human_noise_level = non_human_noise_level
        self.candidate_occluded = candidate_occluded
        self.gaze_self_to_candidate = gaze_self_to_candidate if not candidate_occluded else False
        self.gaze_candidate_to_self = gaze_candidate_to_self
        self.proximity = proximity

        self.candidate_clothing_athletic = False
        self.candidate_clothing_casual = False
        self.candidate_clothing_trendy = False
        self.candidate_clothing_formal = False
        self.candidate_clothing_designer = False
        self.candidate_clothing_eyeglasses = False
        self.candidate_clothing_sunglasses = False
        self.candidate_clothing_luxury_watch = False
        self.candidate_clothing_smart_watch = False
        self.candidate_clothing_hat = False
        self.candidate_clothing_necklace = False
        self.candidate_clothing_rings = False
        self.candidate_clothing_earrings = False

        for item in candidate_clothing:
          if item == "Athletic clothes.":
            self.candidate_clothing_athletic = True
          if item == "Casual clothes.":
            self.candidate_clothing_casual = True
          if item == "Trendy clothes.":
            self.candidate_clothing_trendy = True
          if item == "Formal clothes.":
            self.candidate_clothing_formal = True
          if item == "Designer clothes.":
            self.candidate_clothing_designer = True
          if item == "Eyeglasses.":
            self.candidate_clothing_eyeglasses = True
          if item == "Sunglasses.":
            self.candidate_clothing_sunglasses = True
          if item == "Luxury watch.":
            self.candidate_clothing_luxury_watch = True
          if item == "Smart watch.":
            self.candidate_clothing_smart_watch = True
          if item == "Hat.":
            self.candidate_clothing_hat = True
          if item == "Necklace.":
            self.candidate_clothing_necklace = True
          if item == "Rings.":
            self.candidate_clothing_rings = True
          if item == "Earrings.":
            self.candidate_clothing_earrings = True


In [23]:
GAZE_OPTIONS = ['looking', 'not looking']
HUMAN_CONGESTION_LEVEL_OPTIONS = ['crowded', 'not crowded']
PROXIMITY_OPTIONS = ['nearby', 'far away']
WEATHER_OPTIONS = ['rainy', 'cloudy', 'clear']
HUMAN_NOISE_LEVEL_OPTIONS = ['There are not many people talking', 'There are many people talking']
NON_HUMAN_NOISE_LEVEL_OPTIONS = ['no music is playing', 'music is playing']

def build_scenario(scenario_id, user):
    print(f'Creating scenario for prolific_id: {user.prolific_id}\n')

    # TODO: temporary workaround for removing university and workplace.
    # if "School / University" in user.frequent_locations:
    #   user.frequent_locations.remove("School / University")
    # if "Workplace" in user.frequent_locations:
    #   user.frequent_locations.remove("Workplace")
    # if len(user.frequent_locations) == 0:
    #   user.frequent_locations.append('Sit-down restaurant')

    # Define location based features.
    location = get_random_frequent_location(user.frequent_locations)
    day_of_week, time_of_day, group_size, clothing = get_day_time_group_size_and_clothing(user, location)

    # Define non-location based, deterministic features.
    other_conversational_intensity = 'alone' if (group_size == 'I typically go alone.' or group_size == "I'm typically alone.") else user.listen_or_speak

    # Define non-location based, non-deterministic features.
    gaze_self_to_other = random.choice(GAZE_OPTIONS)
    gaze_other_to_self = random.choice(GAZE_OPTIONS)
    other_occluded = random.choice([True, False])
    human_congestion_level = random.choice(HUMAN_CONGESTION_LEVEL_OPTIONS)
    proximity = random.choice(PROXIMITY_OPTIONS)
    weather = random.choice(WEATHER_OPTIONS)
    human_noise_level = random.choice(HUMAN_NOISE_LEVEL_OPTIONS)
    non_human_noise_level = random.choice(NON_HUMAN_NOISE_LEVEL_OPTIONS)

    """
    # Print location based features.
    print(f'LOCATION: {location}')  # Non-deterministic. *
    print(f'DAY OF WEEK: {day_of_week}')  # Deterministic. *
    print(f'TIME OF DAY: {time_of_day}')  # Deterministic. *
    print(f'GROUP SIZE: {group_size}')  # Deterministic. *
    print(f'CLOTHING: {clothing}')  # Non-deterministic. *
    print('')

    # Print non-location based, deterministic features.
    print(f'AGE: {user.age}')  # *
    print(f'GENDER: {user.gender}')  # *
    print(f'HEIGHT: {user.height}')  # *
    print(f'HAIR TYPE: {user.hair_type}')  # *
    print(f'HAIR COLOR: {user.hair_color}')  # *
    print(f'HAS TATTOOS: {user.has_tattoos}')  # *
    print(f'EDUCATION: {user.education}')  # *
    print(f'IS STUDENT: {user.is_student}')  # *
    print(f'IS IN WORKFORCE: {user.is_in_workforce}')  # *
    print(f'INDUSTRY: {user.industry}')  # *
    print(f'HOBBIES: {user.hobbies}')  # DEPRECATED.
    print(f'FAVORITE HOBBY: {user.favorite_hobby}')  # *
    print(f'INTERESTS: {user.interests}')  # DEPRECATED.
    print(f'FAVORITE INTEREST: {user.favorite_interest}')  # *
    print(f'MUSIC GENRE: {user.music_genre}')  # *
    print(f'PERSONALITY: {user.personality}')  # *
    print(f'OTHER CONVERSATIONAL INTENSITY (based on `listen_or_speak`): {other_conversational_intensity}')  # *
    print(f'SOCIAL MEDIA: {user.social_media}')  # DEPRECATED.
    print(f'FAVORITE SOCIAL MEDIA: {user.favorite_social_media}')  # *
    print(f'MUSIC LISTEN TIME: {user.music_listen_time}')  # *
    print('')

    # Print non-location based, non-deterministic features.
    print(f'GAZE SELF TO OTHER: {gaze_self_to_other}')  # *
    print(f'GAZE OTHER TO SELF: {gaze_other_to_self}')  # *
    print(f'OTHER OCCLUDED: {other_occluded}')  # *
    print(f'HUMAN CONGESTION LEVEL: {human_congestion_level}')  # *
    print(f'OTHER CONVERSATIONAL INTENSITY: {other_conversational_intensity}')  # *
    print(f'PROXIMITY: {proximity}')  # *
    print(f'WEATHER: {weather}')  # *
    print(f'HUMAN NOISE LEVEL: {human_noise_level}')  # *
    print(f'NON HUMAN NOISE LEVEL: {non_human_noise_level}')  # *
    print('')
    """

    # Build scenario info.
    scenario_info = ScenarioInfo(
      scenario_id=scenario_id,
      candidate_prolific_id=user.prolific_id,
      location=location,
      weather=weather,
      human_congestion_level=translate_human_congestion_level(human_congestion_level),
      human_noise_level=translate_human_noise_level(human_noise_level),
      non_human_noise_level=translate_non_human_noise_level(non_human_noise_level),
      candidate_occluded=other_occluded,
      gaze_self_to_candidate=translate_gaze_self_to_candidate(gaze_self_to_other),
      gaze_candidate_to_self=translate_gaze_candidate_to_self(gaze_other_to_self),
      proximity=translate_proximity(proximity),
      candidate_clothing=clothing
    )

    # print(scenario_info.candidate_prolific_id)
    # print(scenario_info.location)
    # print(scenario_info.weather)
    # print(scenario_info.human_congestion_level)
    # print(scenario_info.human_noise_level)
    # print(scenario_info.non_human_noise_level)
    # print(scenario_info.candidate_occluded)
    # print(scenario_info.gaze_self_to_candidate)
    # print(scenario_info.gaze_candidate_to_self)
    # print(scenario_info.proximity)

    # Translate variables to scenario.
    location = translate_location(location)
    day_of_week = translate_day_of_week(day_of_week)
    time_of_day = translate_time_of_day(time_of_day)
    other_conversational_intensity = translate_conversational_intensity(other_conversational_intensity)
    if group_size.endswith('.'):
        group_size = group_size[:-1]  # Remove the period.
    if group_size == 'Over 6 other people':
        group_size = 'more than 6 other people'
    if user.hair_type.endswith('.'):
        user.hair_type = user.hair_type[:-1]  # Remove the period.
    if user.hair_color.endswith('.'):
        user.hair_color = user.hair_color[:-1]  # Remove the period.
    if user.music_listen_time == 'Never.':
        user.music_listen_time = "never listens in public"
    else:
        user.music_listen_time = f"listens {user.music_listen_time.lower()[:-1]} per day in public"

    # Conditional scenario information based on occlusion / gaze.
    verbose_details = False

    if other_conversational_intensity != 'alone':
        other_conversational_intensity = 'with ' + group_size + ' and ' + other_conversational_intensity

    if other_occluded:
        gaze_self_to_other = GAZE_OPTIONS[1]  # Has to be 'not looking'.
        if gaze_other_to_self == GAZE_OPTIONS[0]:  # GAZE_OPTIONS[0] == 'looking'
            gaze_other_to_self = f'A friend mentions that they see a person looking at you who is {proximity}'
        else:
            gaze_other_to_self = f'A friend mentions that they see a person who is {proximity}'
        if other_occluded:
            gaze_other_to_self += ', but you are unable to see them from your angle.'
        else:
            gaze_other_to_self += ', but you choose not to look at them.'

    else:  # other_occluded == false
        gaze_self_to_other = GAZE_OPTIONS[0]  # Hardcode to be 'looking', so that there is an even distribution across the 4 scenario types.
        verbose_details = True
        if gaze_other_to_self == GAZE_OPTIONS[0]:  # GAZE_OPTIONS[0] == 'looking'
            gaze_other_to_self = f'You see a person looking at you who is {proximity}.'
        else:
            gaze_other_to_self = f'You see a person who is {proximity}.'

    verbose_details = True  # Hardcode to verbose.

    details = f"""
Visual Appearance
* Height: {user.height}
* Hair: {user.hair_color} & {user.hair_type}.
* Clothing: {", ".join(clothing).replace('.', '')}.
* Tattoos: {("Yes" if user.has_tattoos == "Yes" else "None")}.

Demographics
* Gender: {user.gender}
* Age: {user.age}.
* Education: {user.education}
* Personality: {user.personality}
* Student status: {("Currently a student" if user.is_student == 'Yes' else "Not currently a student")}.
* Industry: {user.industry[0].upper() + user.industry[1:].lower()}
* Workforce status: {("Currently employed" if user.is_in_workforce == "Yes." else "Not currently employed")}.

Interests & Hobbies
* Favorite hobby: {user.favorite_hobby}
* Favorite interest: {user.favorite_interest}
* Favorite music genre: {user.music_genre} ({user.music_listen_time}).
* Favorite social media: {user.favorite_social_media}."""

    # Print scenario.
    print(f"""
You are at {location} {day_of_week} {time_of_day}.
It is a {weather} day and it's {human_congestion_level}.
{human_noise_level} and {non_human_noise_level}.
{gaze_other_to_self}
The person is {other_conversational_intensity}.

You learn the following information about the person:
{details}""")

    return scenario_info


In [24]:
import os

# write the scenario to a csv file.
def create_scenario_file(filename):
  with open(filename, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow([
      "scenario_id",
      "candidate_prolific_id",
      "location",
      "weather",
      "human_congestion_level",
      "human_noise_level",
      "non_human_noise_level",
      "candidate_occluded",
      "gaze_self_to_candidate",
      "gaze_candidate_to_self",
      "proximity",
      "candidate_clothing_athletic",
      "candidate_clothing_casual",
      "candidate_clothing_trendy",
      "candidate_clothing_formal",
      "candidate_clothing_designer",
      "candidate_clothing_eyeglasses",
      "candidate_clothing_sunglasses",
      "candidate_clothing_luxury_watch",
      "candidate_clothing_smart_watch",
      "candidate_clothing_hat",
      "candidate_clothing_necklace",
      "candidate_clothing_rings",
      "candidate_clothing_earrings"
    ])

def write_to_scenario_file(filename, scenario_info):
  with open(filename, 'a', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow([
      scenario_info.scenario_id,
      scenario_info.candidate_prolific_id,
      scenario_info.location,
      scenario_info.weather,
      scenario_info.human_congestion_level,
      scenario_info.human_noise_level,
      scenario_info.non_human_noise_level,
      scenario_info.candidate_occluded,
      scenario_info.gaze_self_to_candidate,
      scenario_info.gaze_candidate_to_self,
      scenario_info.proximity,
      scenario_info.candidate_clothing_athletic,
      scenario_info.candidate_clothing_casual,
      scenario_info.candidate_clothing_trendy,
      scenario_info.candidate_clothing_formal,
      scenario_info.candidate_clothing_designer,
      scenario_info.candidate_clothing_eyeglasses,
      scenario_info.candidate_clothing_sunglasses,
      scenario_info.candidate_clothing_luxury_watch,
      scenario_info.candidate_clothing_smart_watch,
      scenario_info.candidate_clothing_hat,
      scenario_info.candidate_clothing_necklace,
      scenario_info.candidate_clothing_rings,
      scenario_info.candidate_clothing_earrings
    ])

In [25]:
from datetime import datetime

filename = 'saved_scenarios/' + str(datetime.now()) + '.csv'
create_scenario_file(filename)
scenario_id = 1
for user in selected_users:
    scenario_info = build_scenario(scenario_id, user)
    # Comment out the following line to prevent writing to file.
    # write_to_scenario_file(filename, scenario_info)
    print('\n~~~~~~~~~~\n')
    scenario_id += 1


Creating scenario for prolific_id: 641374b5a30a87f5ed9330f4


You are at the gym on a weekday in the afternoon.
It is a clear day and it's crowded.
There are many people talking and music is playing.
A friend mentions that they see a person who is nearby, but you are unable to see them from your angle.
The person is with 1 other person and they are listening to others in their group.

You learn the following information about the person:

Visual Appearance
* Height: 5 ft 2 in
* Hair: Black & Long (curly / wavy).
* Clothing: Athletic clothes, Smart watch, Necklace.
* Tattoos: Yes.

Demographics
* Gender: Female.
* Age: 24.
* Education: Bachelor's Degree.
* Personality: Introvert.
* Student status: Not currently a student.
* Industry: Social work.
* Workforce status: Not currently employed.

Interests & Hobbies
* Favorite hobby: Games (e.g., board games, video games, puzzles).
* Favorite interest: Health and wellness.
* Favorite music genre: Pop (listens 2 - 3 hours per day in public).
*

Scenarios used in [Part 2] Social interactions study:

ROUND 1 (6 participants): https://pastebin.com/e39ywbaB

ROUND 1 (4 participants): https://pastebin.com/DchBuaQF

ROUND 2 (7 participants): https://pastebin.com/YRrCk5Ur

ROUND 3 (10 participants): https://pastebin.com/PcSw29FW

ROUND 4 (6 participants): https://pastebin.com/705jFjdB